# Dynamic Graph Program

In this notebook we are going to explore the dynamic calls of programs. This feature is particurly important for learning systems and systems with an extensive range of capabilities. If your system have a wide range of capabilities, choosing the one to use in the main program will likely to be impractical because each decision will add latency to your program. In that case, it is better to work with dynamic programs.

### Protected programs

When experimenting with dynamic calls, one important feature of HybridAGI is the fact that we use the dependency graph of HybridAGI to protect the main prompting mechanism that usually contains the safety and learning algorithm of your system. Which means that the system cannot read/search or modify any program that the `main` program depends on (including itself).

In [1]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Program(
    id = "fulfill_objective",
    purpose = "Fulfill the objective",
    program = "fulfill_objective",
))

main.connect("start", "fulfill_objective")
main.connect("fulfill_objective", "end")

main.build()

main.save("data/programs") # We save it into the folder data/programs

print(main)

fulfill_objective = gp.GraphProgram(
    name = "fulfill_objective",
    description = "Try to call an existing program to fulfill the objective",
)

fulfill_objective.add(gp.Action(
    id = "program_search",
    purpose = "Search for relevant routines to fullfil the Objective",
    tool = "GraphProgramSearch",
    prompt = "Use the Objective to describe in ONE short sentence the action to take",
))

fulfill_objective.add(gp.Decision(
    id = "is_routine_known",
    purpose = "Check if the routine to fulfill the objective is in the previous search",
    question = "Is the routine to fulfill the objective in the above search? If you don't know consider the most probable",
))

fulfill_objective.add(gp.Action(
    id = "call_routine",
    purpose = "Pick the most appropriate routine from your context",
    tool = "CallGraphProgram",
    prompt = """
Use the context to known which routine to pick.
Only infer the name of the program without addtionnal details.
Make sure to give only ONE routine name. 
If you don't know which one to pick, try the one with less assumptions.
""",
))

fulfill_objective.connect("start", "program_search")
fulfill_objective.connect("program_search", "is_routine_known")
fulfill_objective.connect("is_routine_known", "call_routine", label = "Yes")
fulfill_objective.connect("is_routine_known", "call_routine", label = "Maybe")
fulfill_objective.connect("is_routine_known", "end", label = "No")
fulfill_objective.connect("call_routine", "end")

fulfill_objective.build()


/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(fulfill_objective:Program {
  id: "fulfill_objective",
  purpose: "Fulfill the objective",
  program: "fulfill_objective"
}),
// Structure declaration
(start)-[:NEXT]->(fulfill_objective),
(fulfill_objective)-[:NEXT]->(end)


In [2]:
# Now let's make some programs to call, first vector only RAG program

search = gp.GraphProgram(
    name = "search",
    description = "Search for information and answer",
)

search.add(gp.Action(
    id = "document_search",
    purpose = "Find relevant documents",
    tool = "DocumentSearch",
    prompt = "Please infer the similarity search query (only ONE item) based on the Objective's question",
))

search.add(gp.Action(
    id = "answer",
    purpose = "Answer the Objective's question",
    tool = "Speak",
    prompt = """
Please answer the Objective's question using the relevant documents in your context.
If no document are relevant just say that you don't know.
Don't state the Objective's question and only give the correct answer.
""",
))

search.connect("start", "document_search")
search.connect("document_search", "answer")
search.connect("answer", "end")

search.build()

search.save("data/programs") # We save it into the folder data/programs

print(search)

// @desc: Search for information and answer
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(document_search:Action {
  id: "document_search",
  purpose: "Find relevant documents",
  tool: "DocumentSearch",
  prompt: "Please infer the similarity search query (only ONE item) based on the Objective's question"
}),
(answer:Action {
  id: "answer",
  purpose: "Answer the Objective's question",
  tool: "Speak",
  prompt: "\nPlease answer the Objective's question using the relevant documents in your context.\nIf no document are relevant just say that you don't know.\nDon't state the Objective's question and only give the correct answer.\n"
}),
// Structure declaration
(start)-[:NEXT]->(document_search),
(document_search)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)


In [3]:
tell_joke = gp.GraphProgram(
    name = "tell_joke",
    description = "Tell a joke to the user",
)

tell_joke.add(gp.Action(
    id = "tell_joke",
    purpose = "Tell a joke",
    tool = "Speak",
    prompt = "Imagine that you are the best comedian on earth, please tell your best joke",
))

tell_joke.connect("start", "tell_joke")
tell_joke.connect("tell_joke", "end")

tell_joke.build()

tell_joke.save("data/programs")

In [4]:
# Now we can embbed them and load them into memory

from hybridagi.memory.integration.local import LocalProgramMemory
from hybridagi.core.pipeline import Pipeline
from hybridagi.core.datatypes import GraphProgramList
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.modules.embedders import GraphProgramEmbedder

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384, # The dimention of the embeddings vector (also called dense vector)
)

prog_pipeline = Pipeline()

prog_pipeline.add("embed_programs", GraphProgramEmbedder(embeddings=embeddings))

embedded_programs = prog_pipeline(GraphProgramList(progs=[main, fulfill_objective, search, tell_joke]))

program_memory = LocalProgramMemory(index_name="dynamic_program")

program_memory.update(embedded_programs)

program_memory.show()

100%|██████████| 4/4 [00:00<00:00, 19.27it/s]


dynamic_program_program_memory.html


In [5]:
# Let's add some documents for the RAG program
from hybridagi.readers import TextReader
from hybridagi.core.pipeline import Pipeline
from hybridagi.modules.splitters import DocumentSentenceSplitter
from hybridagi.modules.embedders import DocumentEmbedder
from hybridagi.memory.integration.local import LocalDocumentMemory

reader = TextReader()

documents = reader("data/SynaLinks_presentation.md")

doc_pipeline = Pipeline()

doc_pipeline.add("chunk_docs", DocumentSentenceSplitter())
doc_pipeline.add("embed_docs", DocumentEmbedder(
    embeddings = embeddings,
))

embedded_documents = doc_pipeline(documents)

document_memory = LocalDocumentMemory(index_name="dynamic_program")

document_memory.update(documents)

100%|██████████| 4/4 [00:00<00:00, 80.29it/s]


In [6]:
import dspy
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents.tools import (
    DocumentSearchTool,
    SpeakTool,
    CallGraphProgramTool,
    GraphProgramSearchTool,
)
from hybridagi.modules.retrievers.integration.local import FAISSDocumentRetriever, FAISSGraphProgramRetriever

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    DocumentSearchTool(
        retriever = FAISSDocumentRetriever(
            document_memory = document_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 0.7,
            k = 5,
            reranker = None,
        ),
    ),
    GraphProgramSearchTool(
        retriever = FAISSGraphProgramRetriever(
            program_memory = program_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 0.7,
            k = 5,
            reranker = None,
        ),
    ),
    CallGraphProgramTool(
        agent_state = agent_state,
        program_memory = program_memory,
    )
]

agent = GraphInterpreterAgent(
    agent_state = agent_state,
    program_memory = program_memory,
    tools = tools
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

result = agent(Query(query="Tell me a joke about neuro-symbolic AI systems"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: Tell me a joke about neuro-symbolic AI systems
--- Step 1 ---
Call Program: fulfill_objective
Program Purpose: Fulfill the objective
--- Step 2 ---
Action Purpose: Search for relevant routines to fullfil the Objective
Action: {
  "query": "\"Find jokes about neuro-symbolic AI systems\"",
  "routines": [
    {
      "name": "tell_joke",
      "description": "Tell a joke to the user",
      "routine": "// @desc: Tell a joke to the user\nCREATE\n// Nodes declaration\n(start:Control {id: \"start\"}),\n(end:Control {id: \"end\"}),\n(tell_joke:Action {\n  id: \"tell_joke\",\n  purpose: \"Tell a joke\",\n  tool: \"Speak\",\n  prompt: \"Imagine that you are the best comedian on earth, please tell your best joke\"\n}),\n// Structure declaration\n(start)-[:NEXT]->(tell_joke),\n(tell_joke)-[:NEXT]->(end)"
    },
    {
      "name": "search",
      "description": "Search for information and answer",
      "routine": "// @desc: Search for informati

We can observe that neither the main progran nor the fullfil objective are present in the search, because they are protected thanks to the dependency graph automatically build.